# 0 - Import and installs

In [5]:
!pip install nnetsauce openml joblib --upgrade --no-cache-dir

In [6]:
import category_encoders as ce
import joblib
import nnetsauce as ns
import numpy as np
import openml
import pandas as pd
import warnings

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from time import time
from functools import lru_cache
from tqdm import tqdm

# 1 Utility functions and global variables

In [7]:
NROWS = 1000
NCOLS = 10

In [8]:
def select_NROWS_NCOLS(X, y):

    def replace_nan_with_median(arr):
      # Calculate the median of each column ignoring NaN values
      median_vals = np.nanmedian(arr, axis=0)

      # Iterate over each column index and replace NaN with the corresponding median value
      for col_idx in range(arr.shape[1]):
          col_values = arr[:, col_idx]
          nan_indices = np.isnan(col_values)
          if np.any(nan_indices):
              col_values[nan_indices] = median_vals[col_idx]

      return arr

    print(f"X.shape (initial): {X.shape}")
    print(f"y.shape (initial): {y.shape}")
    print("Encoding features and response...")
    le = LabelEncoder()
    encoder = ce.HashingEncoder(return_df=False)
    X = np.asarray(encoder.fit_transform(X, y)).astype(np.float32)
    X = replace_nan_with_median(X)
    y = np.asarray(le.fit_transform(y)).astype(np.uint8)
    print("...Done.")
    print(f"Finding top {NCOLS} features if necessary...")
    if X.shape[1] > NCOLS:
        rf = RandomForestClassifier(n_estimators=50, random_state=42)
        rf.fit(X, y)
        indices = np.argsort(rf.feature_importances_)[::-1]
        top_cols = indices[:NCOLS]
        print(f"  top {NCOLS} indices: {top_cols}")
        X = X[:,top_cols]
        print(f"  X reduced shape: {X.shape}")
    print("...Done.")
    if X.shape[0] > NROWS:
      print(f"Subsampling to {NROWS} if necessary...")
      start = time()
      sub = ns.SubSampler(y=y.ravel().astype(np.uint8),
                          n_samples=NROWS, seed=123, n_jobs=-1)
      idx_rows  = sub.subsample().ravel()
      print(f"... Elapsed time for subsampling: {time() - start}")
      print("Number of rows in the subsample: ", len(idx_rows))
      return_X = replace_nan_with_median(X[idx_rows,:])
      return_y = y[idx_rows].ravel().astype(np.uint8)
      print("Done.")
      return return_X, return_y
    else:
      return X, y

# 2 - Get openml cc18 data

In [9]:
warnings.filterwarnings("ignore")

benchmark_suite = openml.study.get_suite('OpenML-CC18') # obtain the benchmark suite

n_tasks = len(benchmark_suite.tasks)

Xys = {}

start = time()
for idx, task_id in tqdm(enumerate(benchmark_suite.tasks)):  # iterate over all tasks
  try: 
    print(f"\n\n task #{idx}/{n_tasks} --------------------")
    warnings.filterwarnings("ignore")
    # Get the task information
    start = time()
    task = openml.tasks.get_task(task_id)  # download the OpenML task
    # get the data from openml
    task_name = task.get_dataset().name
    X_temp, y_temp = task.get_X_and_y()
    Xys[task_name] = {"task_id": task_id,
                      "dataset": select_NROWS_NCOLS(X_temp, y_temp)}
    print(f"Elapsed time for task #{idx}: {time()-start}")
  except:
    continue
print("Total elapsed time: {time()-start}")

0it [00:00, ?it/s]



 task #0/72 --------------------
X.shape (initial): (3196, 36)
y.shape (initial): (3196,)
Encoding features and response...
...Done.
Finding top 10 features if necessary...
  top 10 indices: [20 32  9 14 34  5 31  7  0  6]
  X reduced shape: (3196, 10)
...Done.
Subsampling to 1000 if necessary...


1it [00:01,  1.72s/it]

... Elapsed time for subsampling: 0.3489799499511719
Number of rows in the subsample:  999
Done.
Elapsed time for task #0: 1.716198205947876


 task #1/72 --------------------
X.shape (initial): (20000, 16)
y.shape (initial): (20000,)
Encoding features and response...
...Done.
Finding top 10 features if necessary...


2it [00:04,  2.56s/it]

...

  top 10 indices: [   1 1399    2  351    0 1229 1243  398 1483 1455]
  X reduced shape: (3279, 10)
...Done.
Subsampling to 1000 if necessary...
... Elapsed time for subsampling: 0.023302793502807617
Number of rows in the subsample:  999
Done.
Elapsed time for task #69: 161.80345225334167


 task #70/72 --------------------
X.shape (initial): (3186, 180)
y.shape (initial): (3186,)
Encoding features and response...
...Done.
Finding top 10 features if necessary...


71it [19:18, 117.42s/it]

  top 10 indices: [ 84  89  92 104  99  82  88  93  83  87]
  X reduced shape: (3186, 10)
...Done.
Subsampling to 1000 if necessary...
... Elapsed time for subsampling: 0.02329397201538086
Number of rows in the subsample:  999
Done.
Elapsed time for task #70: 75.77686166763306


 task #71/72 --------------------
X.shape (initial): (5000, 20)
y.shape (initial): (5000,)
Encoding features and response...
...Done.
Finding top 10 features if necessary...


72it [19:29, 16.24s/it] 

  top 10 indices: [ 7  9 19  4 12 10 17 16 15 18]
  X reduced shape: (5000, 10)
...Done.
Subsampling to 1000 if necessary...
... Elapsed time for subsampling: 0.023724079132080078
Number of rows in the subsample:  999
Done.
Elapsed time for task #71: 11.020227193832397
Total elapsed time: {time()-start}


In [ ]:
n_tasks = len(Xys)
print

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

n_tasks = len(Xys)
clf = RandomForestClassifier()

for i, (key, value) in enumerate(Xys.items()):
  print(f"\n\n task #{i+1}/{n_tasks} --------------------")  
  X, y = value["dataset"]
  print(X.shape)
  print(y.shape)
  print(X.dtype)
  print(y.dtype)
  X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                      test_size=0.2,
                                                      random_state=42)
  start = time()
  print(f"accuracy: {clf.fit(X_train, y_train).score(X_test, y_test)}")
  print(f"Elapsed fitting time: {time() - start} seconds")
print(f"Total tasks: {i}")



 task #1/72 --------------------
(999, 10)
(999,)
float32
uint8
accuracy: 0.98
Elapsed fitting time: 0.11992907524108887 seconds


 task #2/72 --------------------
(987, 10)
(987,)
float32
uint8
accuracy: 0.803030303030303
Elapsed fitting time: 0.1830589771270752 seconds


 task #3/72 --------------------
(625, 4)
(625,)
float32
uint8
accuracy: 0.8
Elapsed fitting time: 0.11958980560302734 seconds


 task #4/72 --------------------
(1000, 10)
(1000,)
float32
uint8
accuracy: 0.915
Elapsed fitting time: 0.20526337623596191 seconds


 task #5/72 --------------------
(1000, 10)
(1000,)
float32
uint8
accuracy: 0.89
Elapsed fitting time: 0.22701525688171387 seconds


 task #6/72 --------------------
(699, 9)
(699,)
float32
uint8
accuracy: 0.9642857142857143
Elapsed fitting time: 0.12316179275512695 seconds


 task #7/72 --------------------
(1000, 10)
(1000,)
float32
uint8
accuracy: 0.935
Elapsed fitting time: 0.22740483283996582 seconds


 task #8/72 --------------------
(1000, 6)
(1000,)

# 3 - save data

In [13]:
print(f"Number of data sets: {len(Xys)}")

Number of data sets: 72


In [14]:
joblib.dump(Xys, "openml-cc18-Xys-2024-05-20.pkl")

['openml-cc18-Xys-2024-05-20.pkl']